In [1]:
# Import required Python libraries
import cvxpy as cp
import numpy as np
import numpy.matlib
from scipy import linalg
import matplotlib.pyplot as plt
import ipywidgets as widgets
import scipy 
import scipy.fftpack
from iplabs import IPLabViewer as viewer
import matplotlib
import cv2 as cv
from skimage import color, transform, data, io
import time

# Import module for the project, to evaluate proximal operators
import proximal 

# Configure plotting as dynamic
%matplotlib widget

This notebook is dedicated for experiments/demonstrations on real images or on specially designed data. Note that cells in this notebook can take a **long** time to run on a standard computer. Thus, the results are also stored as `.npy` files. You can go directly to the visualization section of each experiment, that will load the pre-computed results.

# Index
1. [Total Variation](#1.-TV)
    1. [Experiment](#1.A.-Experiment)
    2. [Results](#1.B.-Results)
2. [Hessian-Schatten norm](#2.-Hessian-Schatten-norm)
    1. [Experiment](#2.A.-Experiment)
    2. [Results](#2.B.-Results)
2. [Group Sparsity](#3.-Group-Sparsity)
    1. [Experiment](#3.A.-Experiment)
    2. [Results](#3.B.-Results)

# 1. TV
[Index](#Index)

For a demonstration of $\operatorname{TV}$ regularization, I designed an experiment with the [Shepp-Logan phantom](https://en.wikipedia.org/wiki/Shepp%E2%80%93Logan_phantom), a standard test image in image reconstruction techniques. Moreover, it is a piece-wise constant image, which makes it ideal for $\operatorname{TV}$ regularization. 

## 1.A. Experiment
[Index](#Index)

In the next cell, the Shepp-Logan phantom is loaded from the `scikit-image` data set. Then, white noise is added, sampled from the distribution $\mathcal{N}(0, 0.08)$. To perform the experiment, in order, I evaluate $\mathrm{prox}_{\mathcal{R} + \delta_{\rm \mathbb{R}_+^N}}(\mathbf{v})$, $\mathrm{prox}_{\delta_{\rm \mathbb{R}_+^N}}(\mathrm{prox}_{\mathcal{R}}(\mathbf{v}))$, and $\mathrm{prox}_{\mathcal{R}}(\mathrm{prox}_{\delta_{\rm \mathbb{R}_+^N}}(\mathbf{v}))$. Keeping track of time for reference. Finally, I put the arrays together in a numpy array and store it for future use.

In [2]:
from skimage import color, transform, data
import time
phantom = data.shepp_logan_phantom()
phantom = cv.resize(data.shepp_logan_phantom(), dsize = (200, 200), interpolation = cv.INTER_NEAREST)
noisy_phantom = phantom + np.random.normal(loc=0, scale=0.08, size=phantom.shape)

start = time.time()
opt_nonneg_plus_prox =  proximal.TV_plus_nonneg_prox(noisy_phantom, 0.05)
end = time.time()
print(f'Time taken for prox_(tv + nonneg): {np.round(start - end)} seconds.')

start = time.time()
opt_nonneg_o_prox =  np.maximum(np.zeros_like(noisy_phantom), (proximal.TV_prox(noisy_phantom, 0.05)))
end = time.time()
print(f'Time taken for prox_nonneg(prox_tv): {np.round(start - end)} seconds.')

start = time.time()
opt_prox_o_nonneg = proximal.TV_prox(np.maximum(np.zeros_like(phantom), noisy_phantom), 0.05)
end = time.time()
print(f'Time taken for prox_tv(prox_nonneg): {np.round(start - end)} seconds.')

# Pack original, noisy, ground_truth, prox_nonneg(prox_reg), prox_reg(prox_nonneg), and the error maps in a single numpy array and save array  
phantom_TV_results = np.array([phantom, noisy_phantom, opt_nonneg_plus_prox, opt_nonneg_o_prox, opt_prox_o_nonneg, 
                               np.abs(opt_nonneg_plus_prox - opt_nonneg_o_prox), np.abs(opt_nonneg_plus_prox - opt_prox_o_nonneg)])
np.save('phantom_TV_results.npy', phantom_TV_results)

## 1.B. Results
[Index](#Index)

In the next cell I reaload the data saved previously, unpack it, and plot it. The titles will include the `SNR` in `[dB]` for a better comparison to the original image.

<!-- In the next cell, the Shepp-Logan phantom is loaded from the `scikit-image` data set. Then, white noise is added, sampled from the distribution $\mathcal{N}(0, 0.08)$. Finally, in order, I evaluate $\mathrm{prox}_{\mathcal{R} + \delta_{\rm \mathbb{R}_+^N}}(\mathbf{v})$, $\mathrm{prox}_{\mathcal{R}}(\mathrm{prox}_{\delta_{\rm \mathbb{R}_+^N}}(\mathbf{v}))$ and $\mathrm{prox}_{\delta_{\rm \mathbb{R}_+^N}}(\mathrm{prox}_{\mathcal{R}}(\mathbf{v}))$. Finally, I put the arrays together in a numpy array and store it for future use. -->

In [2]:
# Load results from npy file, and unpack
phantom_TV_results = np.load('phantom_TV_results.npy')
phantom, noisy_phantom, opt_nonneg_plus_prox, opt_nonneg_o_prox, opt_prox_o_nonneg, err_nonneg_o_prox, err_prox_o_nonneg = list(phantom_TV_results) 

# Repack for viewerand declare titles
image_list = [phantom, noisy_phantom, opt_nonneg_plus_prox, opt_nonneg_o_prox, opt_prox_o_nonneg, err_nonneg_o_prox, err_prox_o_nonneg]
title_list = ['Original', f'Noisy (SNR = {np.round(proximal.snr_db(phantom, noisy_phantom), 2)} [dB])',
              '$\mathrm{prox}_{\delta + \mathcal{R}}$' + f' (SNR = {np.round(proximal.snr_db(phantom, opt_nonneg_plus_prox), 2)} [dB])',
              '$\mathrm{prox}_{\delta}(\mathrm{prox}_{\mathcal{R}})$' + f' (SNR = {np.round(proximal.snr_db(phantom, opt_nonneg_o_prox), 2)} [dB])',
              '$\mathrm{prox}_{\mathcal{R}}(\mathrm{prox}_{\delta})$' + f' (SNR = {np.round(proximal.snr_db(phantom, opt_prox_o_nonneg), 2)} [dB])',
              '$\mathrm{prox}_{\delta + \mathcal{R}}$ - $\mathrm{prox}_{\delta}(\mathrm{prox}_{\mathcal{R}})$',
              '$\mathrm{prox}_{\delta + \mathcal{R}}$ - $\mathrm{prox}_{\mathcal{R}}(\mathrm{prox}_{\delta})$']

# Visualize
viewer(image_list, title = title_list)

Button(description='Show Widgets', style=ButtonStyle())

# 2. Hessian-Schatten norm
[Index](#Index)

Experiment designed to test the effect of Hessian-Shatten norm (HS) regularization. In the first part, and for comparison purposes, I also applied the regularization to the Shepp-Logan phantom. However, the piece-wise linear results of HS regularization are not Ideal. So I used another standard, more realistic image, *peppers*. This was taken from the [University of South California Signal and Image Processing Institute](http://sipi.usc.edu/database/database.php?volume=misc&image=13#top).

## 2.A. Experiment
[Index](#Index)

### Shepp Logan Phantom
The following cell reloads the Shepp-Logan phantom, adds noise to it, and calculates the three cases that I am interested in, keeping track of the time for reference. Furthermore, it packs the results and saves them in `.npy` format.
<!-- In the following image I present the results 
* $\mathrm{prox}_{\delta}(\mathrm{prox}_{\mathcal{R}(x)})$: SNR [dB] differs by only 0.1. Stats -rounded to the second decimal- are the exact same
* $\mathrm{prox}_{\mathcal{R}}(\mathrm{prox}_{\delta}(x))$: Results are clearly not the same -->

In [3]:
# phantom = transform.resize(color.rgb2gray(data.astronaut()), (128, 128))
phantom = cv.resize(data.shepp_logan_phantom(), dsize = (100, 100),interpolation = cv.INTER_NEAREST)
noisy_phantom = phantom + np.random.normal(loc=0, scale=0.08, size=phantom.shape)

start = time.time()
opt_nonneg_plus_prox =  proximal.Hessian_Schatten_plus_nonneg_prox(noisy_phantom, 0.05)
end = time.time()
print(f'Time taken for prox_(tv + nonneg): {np.round(start - end)} seconds.')

start = time.time()
opt_nonneg_o_prox =  np.maximum(np.zeros_like(noisy_phantom), (proximal.Hessian_Schatten_prox(noisy_phantom, 0.05)))
end = time.time()
print(f'Time taken for prox_nonneg(prox_tv): {np.round(start - end)} seconds.')

start = time.time()
opt_prox_o_nonneg = proximal.Hessian_Schatten_prox(np.maximum(np.zeros_like(phantom), noisy_phantom), 0.05)
end = time.time()
print(f'Time taken for prox_tv(prox_nonneg): {np.round(start - end)} seconds.')

image_arr = np.array([phantom, noisy_phantom, opt_nonneg_plus_prox, opt_nonneg_o_prox, opt_prox_o_nonneg, 
                      np.abs(opt_nonneg_plus_prox - opt_nonneg_o_prox), np.abs(opt_nonneg_plus_prox - opt_prox_o_nonneg)])
np.save('phantom_Hessian_Schatten.npy', image_arr)

KeyboardInterrupt: 

### Peppers

In the following cell I repeat the experiment for a different image. First I will load the image `peppers_orig`, stored as a `.tiff` file. Moreover, I will resize it to $64\times 64$ (as, with the method I am choosing to solve the proximal operator of the Hessian-Schatten norm, it is unfeasible to try on a larger image) and I will add white noise sampled from the distribution $\mathcal{N}(0, 0.1)$.

In [4]:
peppers_orig = io.imread('peppers.tiff', as_gray = True)
peppers = transform.resize(io.imread('peppers.tiff', as_gray = True), (64, 64), mode='reflect')
noisy_peppers = peppers + np.random.normal(loc=0, scale=0.1, size=peppers.shape)

Now in the next cell I will evaluate $\mathrm{prox}_{\mathcal{R} + \delta_{\rm \mathbb{R}_+^N}}(\mathbf{v})$, $\mathrm{prox}_{\delta_{\rm \mathbb{R}_+^N}}(\mathrm{prox}_{\mathcal{R}}(\mathbf{v}))$, and $\mathrm{prox}_{\mathcal{R}}(\mathrm{prox}_{\delta_{\rm \mathbb{R}_+^N}}(\mathbf{v}))$, and keep track of the time for reference. Finally, I will, save the array as a `.npy` file to avoid losing it when restarting the kernel. 

In [9]:
# Keep track of time
start = time.time()
# Calculate the proximal operator of f = R + δ
opt_nonneg_plus_prox =  proximal.Hessian_Schatten_plus_nonneg_prox(noisy_peppers, 0.05)
end = time.time()
# Print time spent to user 
print(f'Time taken for prox_(tv + nonneg): {np.round(start - end)} seconds.')

# Repeat for  prox_f(prox_g), where g = R and f = δ
start = time.time()
opt_nonneg_o_prox =  np.maximum(np.zeros_like(noisy_peppers), (proximal.Hessian_Schatten_prox(noisy_peppers, 0.05)))
end = time.time()
print(f'Time taken for prox_nonneg(prox_tv): {np.round(start - end)} seconds.')

# Repeat for  prox_f(prox_g), where g = δ and f = R and
start = time.time()
opt_prox_o_nonneg = proximal.Hessian_Schatten_prox(np.maximum(np.zeros_like(noisy_peppers), noisy_peppers), 0.05)
end = time.time()
print(f'Time taken for prox_tv(prox_nonneg): {np.round(start - end)} seconds.')

image_arr = np.array([peppers, noisy_peppers, opt_nonneg_plus_prox, opt_nonneg_o_prox, opt_prox_o_nonneg, np.abs(opt_nonneg_plus_prox - opt_nonneg_o_prox), np.abs(opt_nonneg_plus_prox - opt_prox_o_nonneg)])
np.save('peppers_Hessian_Schatten.npy', image_arr)

Time taken for prox_(tv + nonneg): -9685.0 seconds.
Time taken for prox_nonneg(prox_tv): -8705.0 seconds.
Time taken for prox_tv(prox_nonneg): -9376.0 seconds.


NameError: name 'phantom' is not defined

## 2.B. Results
[Index](#Index)

In this section I will present the results from Hessian-Schatten norm regularization. For a discussion of these results, see section 4 of the [report](../report/Report.pdf). 

### Shepp-Logan phantom
In the following cell, I will load the previously saved results, and plot them with the SNR displayed.

In [4]:
image_arr = np.load('phantom_Hessian_Schatten.npy')

# Unpack from just loaded results
phantom, noisy_phantom, opt_nonneg_plus_prox, opt_nonneg_o_prox, opt_prox_o_nonneg, err_nonneg_o_prox, err_prox_o_nonneg = list(image_arr)

image_list = [phantom, noisy_phantom, opt_nonneg_plus_prox, opt_nonneg_o_prox, opt_prox_o_nonneg, np.abs(opt_nonneg_plus_prox - opt_nonneg_o_prox), np.abs(opt_nonneg_plus_prox - opt_prox_o_nonneg)]
title_list = ['Original', f'Noisy (SNR = {np.round(proximal.snr_db(phantom, noisy_phantom), 3)} [dB])',
              '$\mathrm{prox}_{\delta + \mathcal{R}}$' + f' (SNR = {np.round(proximal.snr_db(phantom, opt_nonneg_plus_prox), 3)} [dB])',
              '$\mathrm{prox}_{\delta}(\mathrm{prox}_{\mathcal{R}})$' + f' (SNR = {np.round(proximal.snr_db(phantom, opt_nonneg_o_prox), 3)} [dB])',
              '$\mathrm{prox}_{\mathcal{R}}(\mathrm{prox}_{\delta})$' + f' (SNR = {np.round(proximal.snr_db(phantom, opt_prox_o_nonneg), 3)} [dB])',
              '$\mathrm{prox}_{\delta + \mathcal{R}}$ - $\mathrm{prox}_{\delta}(\mathrm{prox}_{\mathcal{R}})$',
              '$\mathrm{prox}_{\delta + \mathcal{R}}$ - $\mathrm{prox}_{\mathcal{R}}(\mathrm{prox}_{\delta})$']

viewer(image_list, title = title_list)

Button(description='Show Widgets', style=ButtonStyle())

### Peppers

In the following cell, I will load the previously saved results, and plot them with the SNR displayed.

In [5]:
# Load results
image_arr = np.load('peppers_Hessian_Schatten.npy')

# Unpack from just loaded results
peppers, noisy_peppers, opt_nonneg_plus_prox, opt_nonneg_o_prox, opt_prox_o_nonneg, err_nonneg_o_prox, err_prox_o_nonneg = list(image_arr)
# Create list for image viewer
image_list = [peppers, noisy_peppers, opt_nonneg_plus_prox, opt_nonneg_o_prox, opt_prox_o_nonneg, err_nonneg_o_prox, err_prox_o_nonneg]
# Create title list, calculating SNR up to 3 decimal digits when applicable
title_list = ['Original', f'Noisy (SNR = {np.round(proximal.snr_db(peppers, noisy_peppers), 3)} [dB])',
              '$\mathrm{prox}_{\delta + \mathcal{R}}$' + f' (SNR = {np.round(proximal.snr_db(peppers, opt_nonneg_plus_prox), 3)} [dB])',
              '$\mathrm{prox}_{\delta}(\mathrm{prox}_{\mathcal{R}})$' + f' (SNR = {np.round(proximal.snr_db(peppers, opt_nonneg_o_prox), 3)} [dB])',
              '$\mathrm{prox}_{\mathcal{R}}(\mathrm{prox}_{\delta})$' + f' (SNR = {np.round(proximal.snr_db(peppers, opt_prox_o_nonneg), 3)} [dB])',
              '$\mathrm{prox}_{\delta + \mathcal{R}}$ - $\mathrm{prox}_{\delta}(\mathrm{prox}_{\mathcal{R}})$',
              '$\mathrm{prox}_{\delta + \mathcal{R}}$ - $\mathrm{prox}_{\mathcal{R}}(\mathrm{prox}_{\delta})$']
# Display
viewer(image_list, title = title_list)

Button(description='Show Widgets', style=ButtonStyle())

## 3. Group Sparsity
[Index](#Index)

Since group sparsity by itself is not useful for a simple example like denoising, I have designed a simple, $1$-dimensional experiment to show its effects. 

### 3.A. Experiment
[Index](#Index)

The experiment consists of creating a sequence built from $3$ distinguishable regions, sampled from different normal distributions. Then, *Group Sparsity* is applied to this region, with the $3$ different groups corresponding to the 3 different regions. Since this is not a computationally expensive experiment, the results are not saved, nor the time is taken.

In [7]:
v = np.concatenate([np.random.normal(size = (20, ), scale = 0.2)+0.1, np.random.normal(size = (20, )) + 3, np.random.normal(size = (20, )) + -0.5])

opt_nonneg_plus_prox =  proximal.GS_1D_plus_nonneg_prox(v, 0.5, 3, 2, 1)

# Repeat for  prox_f(prox_g), where g = R and f = δ
opt_nonneg_o_prox =  np.maximum(np.zeros_like(v), (proximal.GS_1D_prox(v, 0.5, 3, 2, 1)))

# Repeat for  prox_f(prox_g), where g = δ and f = R and
opt_prox_o_nonneg = proximal.GS_1D_prox(np.maximum(np.zeros_like(v), v), 0.5, 3, 2, 1)

### 3.B. Results
[Index](#Index)

In the next cell I will plot the results of Group Sparsity with $\|\cdot\|_{2, 1}$ regularization.

In [7]:
plt.figure(figsize = (8, 6))
plt.plot(v, 'o', label = '$\mathbf{v}$', markersize = 6)
plt.plot(opt_nonneg_plus_prox, 's', label = '$\mathrm{prox}_{\mathcal{R} + \delta}(\mathbf{v})$', alpha = 1 ,markersize = 4)
plt.plot(opt_nonneg_o_prox, 'x', label = '$\mathrm{prox}_{\delta}(\mathrm{prox}_{\mathcal{R}}(\mathbf{v}))$', alpha = 1, markersize = 9)
plt.plot(opt_prox_o_nonneg, '+', label = '$\mathrm{prox}_{\mathcal{R}}(\mathrm{prox}_{\delta}(\mathbf{v}))$', alpha = 1, markersize = 11)
plt.axvline(19.5, color='r', linewidth=0.5)
plt.axvline(39.5, color='r', linewidth=0.5)
plt.xlabel('Index', fontname='Calibri', fontsize=14)
plt.ylabel('Value', fontname='Calibri', fontsize=14)
plt.grid()
plt.legend()
plt.title('$||\cdot||_{2, 1}$ Mixed Norm Regularization', fontsize =20)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Pan', 'Pan axes with left…